# Example 1: Simulating a Traffic Network

In this example, we will use the TRACKSIM to simulate vehicle traffic in a SUMO network. For the purposes of this example, we will simulate the charming city of Geldrop, Netherlands which has a population of around 29 thousand people (2023). The corresponding SUMO scenario was created using the osmWebWizard tool from SUMO which is fast way to generate working SUMO scenarios. For more information about osmWebWizard, please refer to the documentation at https://sumo.dlr.de/docs/Tools/Import/OSM.html .

The traffic simulator is handled by the Traffic class in TRACKSIM.

In [ ]:
from tracksim.tracksim import Traffic

help(Traffic)

The traffic class has multiple optional settings which can be configured (please refer to the docstring). However, it is required to give the path to the SUMO configuration file as input. In this example, we will only simulate 1 hour of traffic with a sample interval of 1 second. Additionally, we will enable tracking of the position of each vehicle.

In [ ]:
config_path = 'example_SUMO_scenarios/Geldrop/osm.sumocfg'

traffic = Traffic(config_path, duration=1, time_step=1, record_position=True)

In [ ]:
traffic.simulate_traffic()

Once the simulation has finished, all trip files are found in a directory named 'simulated_trip_files'. The data from each trip is stored in a CSV file. The name of each trip file corresponds to the ID of the vehicle. Let's look at the data for one trip.

In [ ]:
import pandas as pd

trip_file = 'simulated_trip_files/veh0.csv'
trip_data = pd.read_csv(trip_file)

trip_data.info()
trip_data.head()

The trip data contains the time of the simulation, the speed of the vehicle in meters per second and the x and y position of the vehicle in meters.

The path of the vehicle can be plotted.

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-notebook')

plt.figure()
plt.scatter(trip_data['x [m]'], trip_data['y [m]'], c=trip_data['Speed [m/s]']*3.6, cmap='viridis')
plt.colorbar(label='Speed [km/h]')
plt.xlabel('x [m]')
plt.ylabel('y [m]')

Likewise, we can plot the speed of the vehicle over time.

In [ ]:
plt.figure()

plt.plot(trip_data['Time [s]'], trip_data['Speed [m/s]']*3.6) # Convert to km/h
plt.xlabel('Time of simulation [s]')
plt.ylabel('Speed [km/h]')

SUMO has a tendency to give speeds which can vary a lot over a short time. This can lead to high acceleration/deacceleration peaks and potentially unrealistic battery measurements. One way to mitigate this is to low-pass filter the speed profile. This can be done using an exponential moving average.

In [ ]:
from tracksim.utils import exp_average

speed_filtered = exp_average(trip_data['Speed [m/s]'], 0.3)

plt.figure()

plt.plot(trip_data['Time [s]'], trip_data['Speed [m/s]']*3.6, label='Original')
plt.plot(trip_data['Time [s]'], speed_filtered*3.6, label='Filtered')
plt.xlabel('Time of simulation [s]')
plt.ylabel('Speed [km/h]')
plt.legend()